## קוד מבוא

### ספריות

In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### העלאת משתנים להרצת הקוד

In [3]:
path = os.getcwd()

software_root_folder = os.path.dirname(path)

software_folder_location = r'{}\current'.format(software_root_folder)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_folder_location))

forecast_version_folder_location=df_inputs_outputs['location'][1]

sys.path.append(software_root_folder)

## פונקציות

### פונקציות גלובליות

In [4]:
from global_functions import up_load_shp, up_load_df, make_point, up_load_gdb

## 

### העלאת מידע

In [5]:
stat=up_load_shp(r'{}\background_files\stat_11_cbs_2020_with_jtmt_fix.shp'.format(software_folder_location))

In [6]:
stat_in_taz=stat.loc[stat['type_stat']=='in_taz']

stat=stat.loc[stat['type_stat']!='in_taz']

In [7]:
taz=up_load_shp(df_inputs_outputs.loc[4]['location'])

In [8]:
path=r'{}\background_files\BNTL_2022.gdb'.format(software_folder_location)

In [9]:
bld=up_load_gdb(path,'BLDG_Clip')

bld_poi=up_load_gdb(path,'POI_BLDG_Clip')

In [10]:
taz_not_relevant=up_load_df(r'{}\background_files'.format(software_folder_location),'taz_not_relevant_for_pop_2020')

In [11]:
taz_not_relevant=list(taz_not_relevant.Taz_num)

### שכבת בניינים

In [12]:
bld=bld.merge(bld_poi[[ 'FCODE', 'USG_GROUP', 'USG_CODE','USG_SP_NM_LTN','BLDG_ID']],how='left',left_on='UNIQ_ID',right_on='BLDG_ID')

bld=bld.loc[(bld['FCODE_y'].isna())|(bld['USG_CODE']==7600)] #זה קוד לבניינים מעורב שימושים אני מניח שעדיף לשים בניינים מיותרים מאשר הפוך

bld['bld_area']=bld.area

bld_point=make_point(bld).fillna(0)

bld_point.loc[bld_point['HEIGHT']<=0,'HEIGHT']=bld_point['HI_PNT_Z']-bld_point['HT_LAND']

ceiling_height=3

bld_point['bld_m3']=(bld_point['HEIGHT']/ceiling_height).astype(int)*bld_point['bld_area'].astype(int)

bld_point.loc[(bld_point['HEIGHT']-ceiling_height)<=0,'bld_m3']=bld_point['bld_area']

C:\Users\dpere\AppData\Local\Temp\ipykernel_35800\3886493543.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 31.3225      20.58816099  23.03316663 ...  79.56155771  86.00171698
 411.10230836]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  bld_point.loc[(bld_point['HEIGHT']-ceiling_height)<=0,'bld_m3']=bld_point['bld_area']


### הורדת מבנים לא רלוונטים

In [13]:
bld_point=gpd.sjoin(bld_point,taz.loc[(~taz['Taz_num'].isin(taz_not_relevant))&(taz['Palestinia']==0)])

In [14]:
col=['bld_m3','centroid','Taz_num']

bld_point=bld_point[col]

bld_point=gpd.sjoin(bld_point,stat[['STAT','geometry']])

bld_point['bld_m3']=bld_point['bld_m3'].astype(int)

### יצירת מאה אחוז לכל א"ס

In [15]:
sum_bld_m3=pd.pivot_table(bld_point.drop(columns='centroid'),index=['Taz_num','STAT'],aggfunc=sum)[['bld_m3']].reset_index()

C:\Users\dpere\AppData\Local\Temp\ipykernel_35800\3533048335.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  sum_bld_m3=pd.pivot_table(bld_point.drop(columns='centroid'),index=['Taz_num','STAT'],aggfunc=sum)[['bld_m3']].reset_index()


In [16]:
sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat'}).reset_index()

C:\Users\dpere\AppData\Local\Temp\ipykernel_35800\3687277795.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat'}).reset_index()


In [17]:
sum_bld_m3=sum_bld_m3.merge(sum_bld_m3_by_stat,on='STAT')

sum_bld_m3['precent_of_stat_data']=sum_bld_m3['bld_m3']/sum_bld_m3['bld_m3_stat']

sum_bld_m3=sum_bld_m3.loc[(sum_bld_m3['precent_of_stat_data']>0.01)|(sum_bld_m3['Taz_num']==2001)]

In [18]:
sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat_new'}).reset_index()

C:\Users\dpere\AppData\Local\Temp\ipykernel_35800\573162633.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  sum_bld_m3_by_stat=sum_bld_m3.pivot_table(index='STAT',aggfunc=sum)[['bld_m3']].rename(columns={'bld_m3':'bld_m3_stat_new'}).reset_index()


In [19]:
sum_bld_m3=sum_bld_m3.merge(sum_bld_m3_by_stat,on='STAT')

sum_bld_m3['precent_of_stat_data']=sum_bld_m3['bld_m3']/sum_bld_m3['bld_m3_stat_new']

### א"ס ללא פוליגון



In [20]:
stat_in_taz=make_point(stat_in_taz).sjoin(taz[['Taz_num','geometry']])[['STAT','Taz_num']]

stat_in_taz['precent_of_stat_data']=1

### איחוד לטבלה אחת



In [21]:
col=['Taz_num', 'STAT',  'precent_of_stat_data']

precent_of_stat_data=pd.concat([sum_bld_m3[col], stat_in_taz])

In [22]:
precent_of_stat_data.to_excel(r'{}\Intermediates\precent_of_stat_data.xlsx'.format(software_folder_location),index=False)

###  יצירת מידע ברמת אזור תנועה

In [23]:
stat=up_load_df(r'{}\Intermediates'.format(software_folder_location),'stat_cbs_jtmt_2020')

In [24]:
col=['pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

stat[col]=stat[col].multiply(stat['pop'], axis="index")

stat['pop_hardi']=stat['pop']*(stat['pre_hardi']/100)

In [25]:
taz_stat_conver=precent_of_stat_data

In [26]:
col=['STAT',
 'aprt_20',
 'pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop',
 'pop_hardi']

בשכבת האזורי תנועה יהיו רק אזורי תנועה שמפצלים את המידע של למס

In [27]:
taz=pd.merge(taz_stat_conver,stat[col],on='STAT',how='left')

In [28]:
col=['aprt_20',
 'pop_0',
 'pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop',
 'pop_hardi']

In [29]:
taz[col]=taz[col].multiply(taz['precent_of_stat_data'], axis="index")

taz=pd.pivot_table(taz,index='Taz_num',aggfunc=sum)

taz['pre_hardi']=taz['pop_hardi']/taz['pop']

taz['hh_size']=taz['pop']/taz['aprt_20']

C:\Users\dpere\AppData\Local\Temp\ipykernel_35800\196819694.py:3: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  taz=pd.pivot_table(taz,index='Taz_num',aggfunc=sum)


In [30]:
save_taz_path=r'{}\Intermediates'.format(software_folder_location)

In [31]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [32]:
taz.to_excel(r'{}\taz_with_pop_info.xlsx'.format(save_taz_path,file_date))